<a href="https://colab.research.google.com/github/anshupandey/AIOps-EL/blob/main/part3_standard_code_with_mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Quality of Service (QoS) Metrics for 5G Network Optimization

In [ ]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn

In [ ]:
!wget -q https://www.dropbox.com/scl/fi/0rncal8biwhhw5opxyxz6/Quality-of-Service-5G.xlsx?rlkey=gdoq5hw1fgc60khl3equbreyn&st=uzqgtrca&dl=0
!mv Quality-of-Service-5G.xlsx?rlkey=gdoq5hw1fgc60khl3equbreyn Quality-of-Service-5G.xlsx

In [ ]:
# Load the dataset
file_path = 'Quality-of-Service-5G.xlsx'
data = pd.read_excel(file_path, sheet_name='in')
data.shape

(400, 8)

In [ ]:
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation
0,2023-09-03 10:00:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.70
1,2023-09-03 10:00:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.80
2,2023-09-03 10:00:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.90
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85


# Data preprocessing

In [ ]:

# Convert columns to numeric where applicable
data['Signal_Strength'] = data['Signal_Strength'].str.replace(' dBm', '').astype(float)
data['Latency'] = data['Latency'].str.replace(' ms', '').astype(float)
data['Required_Bandwidth'] = data['Required_Bandwidth'].str.replace(' Mbps', '').str.replace(' Kbps', '').astype(float)
data['Allocated_Bandwidth'] = data['Allocated_Bandwidth'].str.replace(' Mbps', '').str.replace(' Kbps', '').astype(float)
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85


# Feature engineering

In [ ]:
# Feature engineering
data['Bandwidth_Efficiency'] = data['Allocated_Bandwidth'] / data['Required_Bandwidth']
data['Signal_Quality_Category'] = pd.cut(data['Signal_Strength'], bins=[-100, -85, -70, 0], labels=['Weak', 'Moderate', 'Strong'])
data.head()

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Bandwidth_Efficiency,Signal_Quality_Category
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70,1.5,Moderate
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80,1.2,Moderate
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75,1.2,Weak
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90,1.5,Moderate
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85,1.5,Moderate


In [ ]:
data.head(20)

,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Bandwidth_Efficiency,Signal_Quality_Category
0,2023-09-03 10:00:00,User_1,Video_Call,-75.0,30.0,10.0,15.0,0.70,1.500000,Moderate
1,2023-09-03 10:00:00,User_2,Voice_Call,-80.0,20.0,100.0,120.0,0.80,1.200000,Moderate
2,2023-09-03 10:00:00,User_3,Streaming,-85.0,40.0,5.0,6.0,0.75,1.200000,Weak
3,2023-09-03 10:00:00,User_4,Emergency_Service,-70.0,10.0,1.0,1.5,0.90,1.500000,Moderate
4,2023-09-03 10:00:00,User_5,Online_Gaming,-78.0,25.0,2.0,3.0,0.85,1.500000,Moderate
5,2023-09-03 10:00:00,User_6,Background_Download,-90.0,50.0,500.0,550.0,0.70,1.100000,Weak
6,2023-09-03 10:00:00,User_7,Web_Browsing,-88.0,30.0,1.0,1.0,0.60,1.000000,Weak
7,2023-09-03 10:00:00,User_8,IoT_Temperature,-95.0,100.0,10.0,15.0,0.50,1.500000,Weak
8,2023-09-03 10:00:00,User_9,Video_Streaming,-82.0,35.0,3.0,3.5,0.80,1.166667,Moderate
9,2023-09-03 10:00:00,User_10,File_Download,-75.0,45.0,2.0,2.0,0.70,1.000000,Moderate


In [ ]:
# Define features and targets
features = ['Application_Type', 'Signal_Strength', 'Required_Bandwidth', 'Allocated_Bandwidth', 'Signal_Quality_Category']
target_latency = 'Latency'
target_efficiency = 'Bandwidth_Efficiency'

In [ ]:
# Handling infinities and large values in the dataset
data = data.replace([np.inf, -np.inf], np.nan)
print(data.shape)
data = data.dropna(subset=['Latency', 'Bandwidth_Efficiency'])
print(data.shape)

(400, 10)
(393, 10)


In [ ]:
# Define target variables
y_latency = data[target_latency]
y_efficiency = data[target_efficiency]

In [ ]:
# Train-test split
X = data[features]
X_train_latency, X_test_latency, y_train_latency, y_test_latency = train_test_split(X, y_latency, test_size=0.2, random_state=42)
X_train_efficiency, X_test_efficiency, y_train_efficiency, y_test_efficiency = train_test_split(X, y_efficiency, test_size=0.2, random_state=42)

In [ ]:
# Preprocessing pipelines
categorical_features = ['Application_Type', 'Signal_Quality_Category']
numerical_features = ['Signal_Strength', 'Required_Bandwidth', 'Allocated_Bandwidth']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Model pipelines
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor),
                                         ('regressor', LinearRegression())]),
    'Random Forest': Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestRegressor(random_state=42))])
}

In [ ]:
# set tracking server
mlflow.set_tracking_uri("http://3.107.14.172:5000/")
mlflow.set_experiment("QoS_Prediction")

2024/12/10 04:33:08 INFO mlflow.tracking.fluent: Experiment with name 'QoS_Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/209821809856878572', creation_time=1733805188099, experiment_id='209821809856878572', last_update_time=1733805188099, lifecycle_stage='active', name='QoS_Prediction', tags={}>

In [ ]:
# Model training and evaluation
results = {}
mlflow.sklearn.autolog()
for name, model in models.items():
  with mlflow.start_run(run_name=name+"Anshu"+str(np.random.randint(1000)),log_system_metrics=True) as run:
    print(f'Training {name} for Latency Prediction...')
    model.fit(X_train_latency, y_train_latency)
    y_pred_latency = model.predict(X_test_latency)

    print(f'Training {name} for Bandwidth Efficiency Prediction...')
    model.fit(X_train_efficiency, y_train_efficiency)
    y_pred_efficiency = model.predict(X_test_efficiency)

    # Store results
    results[name] = {
        'Latency': {
            'MAE': mean_absolute_error(y_test_latency, y_pred_latency),
            'RMSE': np.sqrt(mean_squared_error(y_test_latency, y_pred_latency)),
            'R2': r2_score(y_test_latency, y_pred_latency)
        },
        'Efficiency': {
            'MAE': mean_absolute_error(y_test_efficiency, y_pred_efficiency),
            'RMSE': np.sqrt(mean_squared_error(y_test_efficiency, y_pred_efficiency)),
            'R2': r2_score(y_test_efficiency, y_pred_efficiency)
        }
    }

# Display results
results

2024/12/10 04:37:11 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/10 04:37:11 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Linear Regression for Latency Prediction...
Training Linear Regression for Bandwidth Efficiency Prediction...
🏃 View run Linear RegressionAnshu694 at: http://3.107.14.172:5000/#/experiments/209821809856878572/runs/539b9fbfa88a40f983f86dc9436f4438
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/209821809856878572


2024/12/10 04:37:31 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/10 04:37:31 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/12/10 04:37:32 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/10 04:37:32 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Random Forest for Latency Prediction...
Training Random Forest for Bandwidth Efficiency Prediction...
🏃 View run Random ForestAnshu172 at: http://3.107.14.172:5000/#/experiments/209821809856878572/runs/a7cf201b4cde4d93913332df07b08093
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/209821809856878572


2024/12/10 04:38:17 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/10 04:38:17 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


{'Linear Regression': {'Latency': {'MAE': 6.365094420185795,
   'RMSE': 7.946033854829205,
   'R2': 0.8076767901442934},
  'Efficiency': {'MAE': 0.1350199047317324,
   'RMSE': 0.21662304096277932,
   'R2': 0.00029450045440848616}},
 'Random Forest': {'Latency': {'MAE': 1.8966356238698008,
   'RMSE': 4.429782018774095,
   'R2': 0.9402283026952916},
  'Efficiency': {'MAE': 0.03321397254334228,
   'RMSE': 0.09458618553685297,
   'R2': 0.8094020574968452}}}

In [ ]:
eval_data = X_test_latency
eval_data['target'] = y_test_latency

In [ ]:
mlflow.set_experiment("Latency_Prediction")

2024/12/10 05:02:36 INFO mlflow.tracking.fluent: Experiment with name 'Latency_Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/959328149099913042', creation_time=1733806956605, experiment_id='959328149099913042', last_update_time=1733806956605, lifecycle_stage='active', name='Latency_Prediction', tags={}>

In [ ]:
# Model pipelines
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor),
                                         ('regressor', LinearRegression())]),
    'Random Forest': Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestRegressor(random_state=42,max_depth=12,min_samples_leaf=3))])
}

In [ ]:
mlflow.sklearn.autolog()
for name, model in models.items():
  run_name = f"Latency {name} Anshu"+str(np.random.randint(1000))
  with mlflow.start_run(run_name=run_name,log_system_metrics=True) as run:
    print(f'Training {name} for Latency Prediction...')
    model.fit(X_train_latency, y_train_latency)

    mlflow.sklearn.log_model(model,run_name)
    model_uri = mlflow.get_artifact_uri(run_name)
    y_pred_latency = model.predict(X_test_latency)

    result = mlflow.evaluate(model_uri,eval_data,targets='target',model_type='regressor')

2024/12/10 05:05:38 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/12/10 05:05:38 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training Linear Regression for Latency Prediction...


2024/12/10 05:05:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/10 05:05:55 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/12/10 05:05:55 WARNING mlflow.models.evaluation.evaluators.shap: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


🏃 View run Latency Linear Regression Anshu506 at: http://3.107.14.172:5000/#/experiments/959328149099913042/runs/16fbb0d6a0c64d02839ece333959f939
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/959328149099913042


...

Training Random Forest for Latency Prediction...


2024/12/10 05:06:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/12/10 05:06:25 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/12/10 05:06:25 WARNING mlflow.models.evaluation.evaluators.shap: Skip logging model explainability insights because the shap explainer None requires all feature values to be numeric, and each feature column must only contain scalar values.


🏃 View run Latency Random Forest Anshu864 at: http://3.107.14.172:5000/#/experiments/959328149099913042/runs/1de9fbde5a7248be8ce2b70e8bba4dc7
🧪 View experiment at: http://3.107.14.172:5000/#/experiments/959328149099913042


2024/12/10 05:06:26 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/10 05:06:27 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
